<a href="https://colab.research.google.com/github/NiloyPurkait/cognitive-libraries/blob/master/Data%20processing/Clean_company_names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding similar company name and auto matching them

This script will use NLP and ML technique to match similar company names. Matching form common words like "International" and "Corporation" as well as business forms like "BVBA" will be discounted  in the algorithm.

Library used:
* pandas
* fuzzywuzzy

In [ ]:
! pip install fuzzywuzzy

In [ ]:
import math
import pandas as pd
import numpy as np
import spacy
import re
import fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from collections import Counter
import random

random.seed(13)

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the table in pandas DataFrame

Here we load in the downloaded version of the Google Sheets file on drive and load it into pandas. 

Here it is important that the CSV file of the database in in the exact location specified below (drive/My Drive/Colab Notebooks) and that the name is exactly similar, so 'Database - Active Campaign Contacts - Database - Active Campaign Contacts.csv'

In [ ]:
file_name = 'drive/My Drive/Colab Notebooks/Database - Active Campaign Contacts - Database - Active Campaign Contacts.csv'



pd.set_option('display.max_columns', 500)
df_dirty = pd.read_csv(file_name, sep = ',')
df_dirty.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (14,54) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,Email,First Name,Last Name,Phone Number,Date Created,IP Address,User Agent,Organization,Account (previously Organization),Account ID,Job Title,*Vacature,*Location,*Zip Code,*Source,*Link naar vacature,*Street & Nr,*VAT/BTW,*Job title,*Mobile Phone,*Phone (ingegeven door prospect),*Alternative Email,*Website URL,*Personalised video link,*utm_source,*utm_medium,*utm_campaign,*utm_term,*Video link,*Type recruiter,*utm_content,*Can,*State,*Procedure,*Marketing,*Function,*Company,*Akkoord met privacy,*Is het bedrijf werkzaam in rekrutering?,*LinkedIn URL,*UTM Source,*UTM Medium,*UTM Campaign,*UTM Term,*UTM Content,*Blitzlink,*AC URL,*Op VDAB?,*Company size,*Sector information,*Opted-in Nurturing,*LinkedIn Post,*Calendly event,*Hidden field,*Prospect Scoring,*Score 3,Tags
0,39330,000bisnode@fakemail.com,Dorien,De Vos,0484 557158,2020-08-19 8:10:50,0.0.0.0,NaN,DSV Solutions,DSV Solutions,2903.0,Corporate Recruitment Specialist Belux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,~|,Bisnode contacts - Jo - Fake mail - 19/08/20
1,31355,01info@dekorenbloem.net,Evelien,NaN,NaN,2020-06-02 9:25:33,0.0.0.0,NaN,De Korenbloem,De Korenbloem,15533.0,NaN,wooncoach,KORTRIJK,NaN,VDAB,https://www.vdab.be/vindeenjob/vacatures/61116...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,~|,"Raas - Sales Tone Of Voice,Raas Short - Body d..."
2,38539,100bisnode@fakemail.be,Inge,Rooyakkers,+32 11 440460,2020-08-11 3:32:32,0.0.0.0,NaN,Beltaste Hamont,Beltaste Hamont,12688.0,Human Resource Manager,NaN,NaN,3930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van producten van vlees of van v...,NaN,NaN,NaN,NaN,0,~|,Bisnode
3,38540,101bisnode@fakemail.be,Laura,Snyders,+32 89 460360,2020-08-11 3:32:32,0.0.0.0,NaN,IZICO BOCHOLT,IZICO BOCHOLT,17655.0,Human Resource Manager,NaN,NaN,3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van producten van vlees of van v...,NaN,NaN,NaN,NaN,0,~|,Bisnode
4,38541,102bisnode@fakemail.be,Philip,Lenders,+32 89 353181,2020-08-11 3:32:32,0.0.0.0,NaN,Spronken Orthopedie,Spronken Orthopedie,3988.0,Human Resource Manager,NaN,NaN,3600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van medische en tandheelkundige ...,NaN,NaN,NaN,NaN,0,~|,Bisnode


In [ ]:
col_name= 'Organization'

# Manual stopword list
extra_keywords = [
                  'constructions',
                  'construction',
                  'construct',
                  'ingenieursbureau',
                  'antwerpen',
                  'gemeente',
                  'advocatenkantoor',
                  'recruitment',
                  'nv',
                  'abvv',
                  'bvba',
                  'bv',
                  'vof',
                  'bv',
                  'gcv',
                  'cva',
                  'gemeente',
                  'vzw',
                  'holding',
                  'corporation',
                  'groep',
                  'international',
                  'belgium',
                  'france',
                  'team',
                  'internetmarketing',
                  'vilvoorde',
                  'vastgoed',
                  'bouwbedrijf',
                  'personeelsmanagement',
                  'overheidspersoneel',
                  'cars',
                  'food',
                  'leuven',
                  'builders',
                  'intermediagroep',
                  'media',
                  'jobs'      
                  ]




In [ ]:


def get_stopwords(df, col_name, extra_kws = extra_keywords):
  all_names = df[col_name].unique()
  names_freq = Counter()
  for name in all_names:
      names_freq.update(str(name).split(" "))
  freq_ws = [word.lower() for ( word, _ ) in names_freq.most_common(30)] 
  # Frequency based stopword list
  return freq_ws + [i for i in extra_kws if i not in freq_ws]
  
stopwords = get_stopwords(df_dirty, col_name)

In [ ]:
# All stopwords

print(len(stopwords))
print(stopwords)

63
['de', '&', 'van', 'belgium', 'group', '-', 'en', 'services', 'international', 'europe', 'solutions', 'benelux', 'the', 'transport', 'groep', 'logistics', 'het', 'consulting', 'centrum', 'service', 'company', 'nederland', 'partners', 'co', 'hotel', 'systems', 'antwerpen', 'garage', 'construct', 'voor', 'constructions', 'construction', 'ingenieursbureau', 'gemeente', 'advocatenkantoor', 'recruitment', 'nv', 'abvv', 'bvba', 'bv', 'vof', 'bv', 'gcv', 'cva', 'gemeente', 'vzw', 'holding', 'corporation', 'france', 'team', 'internetmarketing', 'vilvoorde', 'vastgoed', 'bouwbedrijf', 'personeelsmanagement', 'overheidspersoneel', 'cars', 'food', 'leuven', 'builders', 'intermediagroep', 'media', 'jobs']


## Data Preprocessing

The first step is to preprocess the data in the column 'Organization'

To make sure inconsistent database entries will be mathced in an optimal way, all names in the column will be lower cased, stripped of trailing and leading spaces, and point kommas and dashed will be removed. By doins so, matching will be better and easier. 

## Frequency of words
Since we have lots of companies, we want our model to discount most frequent words so that it will not focus the match on words like 'Corporation' or 'bvba'

First we find the 30 most common words in all company names. As we will be expecting them to be repeating a lot even in companies that is not the same, we cannot match company names using them. The way we do it is we will deduct the matching score of a pair if any keywords is present in the names.

In [ ]:
df_dirty[col_name].value_counts()

Mercedes-Benz           54
Colruyt Group           33
Belfius                 25
NMBS                    25
D'Ieteren               25
                        ..
Ethilog                  1
Menerga                  1
Lafas                    1
Hestia Houtatelier       1
Nedschroef Herentals     1
Name: Organization, Length: 18322, dtype: int64

In [ ]:
class NameCleaner:
  def __init__(self, stopwords, col_name):

    self.stopwords = stopwords
    self.col_name = col_name

  def is_ascii(self, s):
      return all(ord(c) < 128 for c in s)

  def clean_special_characters(self, txt):
    seps = [" ",";",":","@",".",",","*","#","|","/","\\","-","_","?","%","!","^","(",")"]
    default_sep = seps[0]
    for sep in seps[1:]:
      txt = txt.replace(sep, default_sep)
    re.sub(' +', ' ', txt)
    temp_list = [i.strip() for i in txt.split(default_sep)]
    temp_list = [i for i in temp_list if i]
    return " ".join(temp_list)

  def clean_stopword(self, txt):
    temp_list = txt.split(" ")
    temp_list = [i for i in temp_list if i.lower() not in self.stopwords]
    return " ".join(temp_list)

  def __call__(self, data, dropForeign = True):
    data.dropna(subset=[self.col_name], inplace=True)
    data = data.rename_axis('CompanyID').reset_index()
    data['nonAscii_count'] = data[self.col_name].apply(lambda x: sum([not self.is_ascii(c) for c in x]))
    if dropForeign:
      data = data[data.nonAscii_count==0]
    else:
      pass
    data.drop('nonAscii_count', axis=1, inplace=True)
    data_clean = data.copy()
    data_clean['CustomerName_clean'] = data_clean[self.col_name].apply(lambda x: x. lower())
    data_clean['CustomerName_clean'] = data_clean['CustomerName_clean'].apply(self.clean_special_characters)
    data_clean['CustomerName_clean'] = data_clean['CustomerName_clean'].apply(self.clean_stopword)
    return data_clean



In [ ]:
name_cleaner =  NameCleaner(stopwords=stopwords, col_name=col_name)

df = name_cleaner(df_dirty)

In [ ]:
df.head()

,CompanyID,ID,Email,First Name,Last Name,Phone Number,Date Created,IP Address,User Agent,Organization,Account (previously Organization),Account ID,Job Title,*Vacature,*Location,*Zip Code,*Source,*Link naar vacature,*Street & Nr,*VAT/BTW,*Job title,*Mobile Phone,*Phone (ingegeven door prospect),*Alternative Email,*Website URL,*Personalised video link,*utm_source,*utm_medium,*utm_campaign,*utm_term,*Video link,*Type recruiter,*utm_content,*Can,*State,*Procedure,*Marketing,*Function,*Company,*Akkoord met privacy,*Is het bedrijf werkzaam in rekrutering?,*LinkedIn URL,*UTM Source,*UTM Medium,*UTM Campaign,*UTM Term,*UTM Content,*Blitzlink,*AC URL,*Op VDAB?,*Company size,*Sector information,*Opted-in Nurturing,*LinkedIn Post,*Calendly event,*Hidden field,*Prospect Scoring,*Score 3,Tags,CustomerName_clean
0,0,39330,000bisnode@fakemail.com,Dorien,De Vos,0484 557158,2020-08-19 8:10:50,0.0.0.0,NaN,DSV Solutions,DSV Solutions,2903.0,Corporate Recruitment Specialist Belux,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,~|,Bisnode contacts - Jo - Fake mail - 19/08/20,dsv
1,1,31355,01info@dekorenbloem.net,Evelien,NaN,NaN,2020-06-02 9:25:33,0.0.0.0,NaN,De Korenbloem,De Korenbloem,15533.0,NaN,wooncoach,KORTRIJK,NaN,VDAB,https://www.vdab.be/vindeenjob/vacatures/61116...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,~|,"Raas - Sales Tone Of Voice,Raas Short - Body d...",korenbloem
2,2,38539,100bisnode@fakemail.be,Inge,Rooyakkers,+32 11 440460,2020-08-11 3:32:32,0.0.0.0,NaN,Beltaste Hamont,Beltaste Hamont,12688.0,Human Resource Manager,NaN,NaN,3930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van producten van vlees of van v...,NaN,NaN,NaN,NaN,0,~|,Bisnode,beltaste hamont
3,3,38540,101bisnode@fakemail.be,Laura,Snyders,+32 89 460360,2020-08-11 3:32:32,0.0.0.0,NaN,IZICO BOCHOLT,IZICO BOCHOLT,17655.0,Human Resource Manager,NaN,NaN,3950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van producten van vlees of van v...,NaN,NaN,NaN,NaN,0,~|,Bisnode,izico bocholt
4,4,38541,102bisnode@fakemail.be,Philip,Lenders,+32 89 353181,2020-08-11 3:32:32,0.0.0.0,NaN,Spronken Orthopedie,Spronken Orthopedie,3988.0,Human Resource Manager,NaN,NaN,3600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bisnode,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100 - 199,Vervaardiging van medische en tandheelkundige ...,NaN,NaN,NaN,NaN,0,~|,Bisnode,spronken orthopedie


In [ ]:
[i for i in list(df['CustomerName_clean']) if 'dematic' in i]

['dematic', 'dematic', 'dematic']


## Fuzzy Matching
Here for each group, we use `fuzzywuzzy.token_sort_ratio` to matching the names. Different form the basic `fuzzywuzzy.ratio` which use Levenshtein Distance to calculate the differences, it allow the token (words) in a name to swap order and still give a 'perfect' match. 

In [ ]:
class Org_Name_Matches:

  def __init__(self, company, df, col_name):
    self.company = company

    self.org_list = df[col_name].unique()
    self.matches = fuzzywuzzy.process.extract(company, 
                                              self.org_list,
                                              limit=10,
                                              scorer=fuzzywuzzy.fuzz.token_sort_ratio)

  def get_match_scores(self):
    return self.matches

  def get_score_for(self, company):
    if company in [i[0] for i in self.matches]:
        try:
            name, score = [(str(i[0]),i[1]) for i in self.matches if i[0] == company][0]
            if (len(name.split()) < 2): 
              score = score - 4
            return score
        except Exception as e:
            return self.matches

  def get_uncertain_matches(self, lower_thresh=80, higher_thresh=90):
    
    return [(i[0], i[1]) for i in self.matches if lower_thresh < i[1] < higher_thresh]

  def get_known_aliases(self, higher_thresh=90):
    
    return [(i[0], i[1]) for i in self.matches if higher_thresh <= i[1] ]

  def get_most_freq_name(self, lower_thresh=90):

    all_org_list = list(df[col_name])

    matches = [(i[0], i[1]) for i in self.matches if lower_thresh < i[1] ]
   
    if not matches :
      return self.company

    org_counts = [(j[0], all_org_list.count(j[0])) for j in matches]
    return [k[0] for k in org_counts if k[1] == max(l[1] for l in org_counts)][0]

In [ ]:

###TESTS
def test_matches_for(name):
  tempo_match = Org_Name_Matches(name, df, 'CustomerName_clean')
  print('-| Predicted alias : \n      ',
        tempo_match.get_most_freq_name())

  print('\n-| Top 5 : ')
  for i in tempo_match.get_match_scores()[:5]:
    print('     ', i)


In [ ]:
test_matches_for('dematic nv')

-| Predicted alias : 
       dematic nv

-| Top 5 : 
      ('dematic', 82)
      ('detic', 67)
      ('deman', 67)
      ('demcon', 62)
      ('pvs electromatic', 62)


In [ ]:
#dematic, dematic nv
#activak, activak vzw
#activa 
#delllebron, wzcdellebron 
#fashion club 70, fashion club nv

## only if seperate word, strip business type


## Running the script and creating an output file where the name changes are listed on

Here, the main code is created. This code will loop through every organization name and finds matches for alias. Matches are based on the similarity score, which is also listed in the output file. For a score > 93, the name will be replaced by the alias found by the model. If score is between 80 - 93, it will give an 'uncertain match' wich can be used to save time when manually checking! 

In [ ]:
### MAIN CODE THAT PRODUCES ALIAS FILE

def process_dataset(df, col_name):
  col_name = 'CustomerName_clean'
  # get unique names
  all_names = np.array([str(i) for i in list(df[col_name].unique())])

  # create dataframe to track aliases
  alias_dataset = pd.DataFrame(columns=['names',
                                        'alias',
                                        'score',
                                        'top 5 matches',
                                        'uncertain matches'])
  all_names.sort()
  # Set names to all unique names
  alias_dataset['names'] = all_names 

  # for each unique name
  for index, row in alias_dataset.iterrows():
    company = row['names']

    # get matches from dataset
    matches = Org_Name_Matches(str(company), df, col_name)
    possible_match = matches.get_most_freq_name()
    
    # record matches & score in alias dataframe
    row['top 5 matches'] = matches.get_match_scores()[:5]
    row['alias'] = possible_match
    row['score'] = matches.get_score_for(possible_match)
    row['uncertain matches'] = matches.get_uncertain_matches()

    if index%20==0:
      prog=round(index/len(alias_dataset)*100, 2)
      print('Progress saved at: ', prog, '%')


  alias_dataset.to_csv('drive/My Drive/Colab Notebooks/alias_file_new1.csv') ## location of output file!
  return alias_dataset







## CHECK THE OUTPUT FILE, CHANGE POTENTIAL MISTAKES AND GO FURTHER BELOW. DONT RUN CODE WITHOUT CHECKING OUTPUT FILE

Now, the alias file is created and located in the same 'Colab Notebooks' folder on drive. This file can be used to see the changes the script wants to made. In case of mistakes, these can be corrected here. After checking, we will load in the alias file and apply changes to the original database file. 

- From the output file, aliases with score > 90 will be automatically taken as alias
- In case of good alias with score lower than 90 --> change score to 100 manually
- In case wrong alias with score above 90 --> change score to 0 manually
- Script below will also alias the manually changed codes

In [ ]:

# to run whole workflow from scratch
from_scratch = True

In [ ]:
## make sure output file is stored in Drive/MyDrive/Colab Notebook, as .xlsx file!
if from_scratch:
  test_df = process_dataset(df, col_name='CustomerName_clean')
else:
  test_file_name ='drive/My Drive/Colab Notebooks/alias_file_new.csv'  ## make sure xlsx
  test_df = pd.read_excel(test_file_name)
  print('Loaded alias file.')


Progress saved at:  0.0 %
Progress saved at:  0.11 %
Progress saved at:  0.23 %
Progress saved at:  0.34 %
Progress saved at:  0.46 %
Progress saved at:  0.57 %
Progress saved at:  0.68 %
Progress saved at:  0.8 %
Progress saved at:  0.91 %
Progress saved at:  1.03 %
Progress saved at:  1.14 %
Progress saved at:  1.25 %
Progress saved at:  1.37 %
Progress saved at:  1.48 %
Progress saved at:  1.6 %
Progress saved at:  1.71 %
Progress saved at:  1.82 %
Progress saved at:  1.94 %
Progress saved at:  2.05 %
Progress saved at:  2.17 %
Progress saved at:  2.28 %
Progress saved at:  2.4 %
Progress saved at:  2.51 %
Progress saved at:  2.62 %
Progress saved at:  2.74 %
Progress saved at:  2.85 %
Progress saved at:  2.97 %
Progress saved at:  3.08 %
Progress saved at:  3.19 %
Progress saved at:  3.31 %
Progress saved at:  3.42 %
Progress saved at:  3.54 %
Progress saved at:  3.65 %
Progress saved at:  3.76 %
Progress saved at:  3.88 %
Progress saved at:  3.99 %
Progress saved at:  4.11 %
Progr

In [ ]:
# Add exceptions 
exception_names = ['a s p',
                   'a s w',
                   'antwerpen']
                   #'lambrecht',
                   #'wijnants',
                   #'dge construction',
                   #'integral',
                   #'youca',
                   #'u&i learning']


In [ ]:

def get_changes(df, test_df, col_name):

  update_dic = {}

  for index, row in test_df.iterrows():
    company = str(row['names']).strip().replace('  ', ' ')
    alias = str(row['alias']).strip().replace('  ', ' ')
    score = row['score'] 
    try:
      if (company != alias) and (int(score)>=90):
        update_dic.update({company : [alias, score]}) 
        ### add if statement that gives back company name if there's no alias. ##
      else:
        update_dic.update({company : [company, 102]}) 
    except Exception as e:
      print(' '*10,' -| ERROR', e)
      pass
  return update_dic

def remove_exceptions(exception_names, updates):
  for i in exception_names:
    updates.pop(i, None)
  return updates

def prepare_changes(df, test_df, exception_names, col_name='Organization'):
  updates = get_changes(df, test_df, col_name)
  return remove_exceptions(exception_names, updates)


def update_changes(df, test_df, exception_names, col_name='Organization'):
  df['alias'] = None
  updates = prepare_changes(df, test_df, exception_names)
  print('   -> Changes:')
  for index, row in df.iterrows():
    company =  str(row[col_name]).replace(' ', ' ').strip()
    if company in updates.keys():
      df.loc[index, 'alias'] = updates[company][0]
      print( '-|', company,' ---> ', df.loc[index, 'alias'])
  return df

final_df = update_changes(df, test_df, exception_names)
final_df.to_csv('Cleaned Database Active Clients.csv')

Streaming output truncated to the last 5000 lines.
-| mazars  --->  mazars
-| milieu service nederland  --->  milieu service nederland
-| mn  --->  mn
-| mitsubishi turbocharger and engine europe  --->  mitsubishi turbocharger and engine europe
-| academisch ziekenhuis maastricht  --->  academisch ziekenhuis maastricht
-| neopost technologies  --->  neopost technologies
-| ngis  --->  ngis
-| yokogawa electric corporation  --->  yokogawa electric corporation
-| oleon  --->  oleon
-| peri  --->  peri
-| pitpoint  --->  pitpoint
-| productlife group  --->  productlife group
-| promo sapiens  --->  promo sapiens
-| promoyo  --->  promoyo
-| prowise  --->  prowise
-| pycogroup  --->  pycogroup
-| koninklijke belgische voetbalbond  --->  koninklijke belgische voetbalbond
-| renewi  --->  renewi
-| berry bramlage  --->  berry bramlage
-| salesup  --->  salesup
-| saxion  --->  saxion
-| scania belgium  --->  scania belgium
-| scan sys  --->  scan sys
-| sharp services  --->  sharp services
-

In [ ]:
a= final_df['alias'].value_counts()
a

nan                              3739
mercedes benz                      56
colruyt group                      33
                                   28
d'ieteren                          25
                                 ... 
ckg sint clara                      1
vind! brugge                        1
vanallier                           1
hendrickx feesten                   1
metis instruments & equipment       1
Name: alias, Length: 18038, dtype: int64

In [ ]:
len(a)

18038

In [ ]:
[i for i in final_df.alias if i!=None]

['dethier bouwbedrijf',
 'mathys martin',
 'stanley black & decker inc',
 'merck sharp & dohme',
 'glaxosmithkline',
 'cryns carrosserie center',
 'vrije universiteit brussel',
 'dixon & company',
 'elmos expert',
 'qualiphar',
 '',
 'goodlife foods',
 'interieurbouw van laer',
 '',
 'dienst persoonlijk assistent',
 '',
 'metsense',
 'clearxperts',
 'maandag',
 'schapmans camps',
 '',
 'lambrechts',
 'search & selection',
 "pietercil delby's",
 '',
 'volvo cars',
 'sint  monica',
 'huize sint jozef',
 'viabuild!',
 'wolfs laboratoria',
 'caw halle  vilvoorde',
 'bofrost',
 '',
 'aqtor!',
 'buro ii archi+i',
 'bouwonderneming damman',
 'e d &a',
 'lambrechts',
 'michielsen',
 'michielsen',
 'yde bart',
 'vlaamse overheid (agentschap overheidspersoneel)',
 'helder! personeelsmanagement',
 'woon en zorgcentrum denderrust',
 '',
 '',
 'ado icarus',
 'buro nexus',
 'toi toi & dixi',
 'woon en zorgcentrum denderrust',
 '',
 'hcm vision',
 '',
 'bricks and leisure vastgoed',
 'woon en zorgcen